In [39]:
pip install pandas numpy matplotlib tensorflow


You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


In [41]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"FACT_CUSTOMER_ORDERS"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [42]:
df = df[['PRODUCT_ID','ORDER_DATE','ORDER_QTY']]

In [34]:
df['ORDER_DATE']=pd.to_datetime(df['ORDER_DATE'])

In [35]:
df = df.sort_values('ORDER_DATE')